# Download Data

### CPI

In [1]:
from fredapi import Fred
import pandas as pd
from dateutil.relativedelta import relativedelta
import datetime

# Replace with your valid API key
fred = Fred(api_key='333b82f3ea3ee8858ff976082f2b5236')

# Set up dates
today = datetime.datetime.today()
start_date = today - relativedelta(years=25)
end_date = today

# Define countries and their tickers
cpi_tickers = {
    "USD CPI": "USACPIALLMINMEI",
    "EUR CPI": "CP0000EZ19M086NEST",
    "GBP CPI": "GBRCPIALLMINMEI",
    "CHF CPI": "CHECPIALLMINMEI",
    "CAD CPI": "CANCPIALLMINMEI",
}

# Fetch CPI data for each country
cpi_data = {}
for country, ticker in cpi_tickers.items():
    try:
        cpi_data[country] = fred.get_series(ticker)
    except ValueError as e:
        print(f"Error fetching data for {country} ({ticker}): {e}")

# Convert to DataFrames
cpi_dataframes = {}
for country, data in cpi_data.items():
    cpi_dataframes[country] = pd.DataFrame(data, columns=[country])

# Filter data for Q1 2000 to today
for country, df in cpi_dataframes.items():
    cpi_dataframes[country] = df.loc[start_date:end_date]

# Merge all DataFrames into a single DataFrame
cpi_df = pd.concat(cpi_dataframes.values(), axis=1)

# Transform the index to 'YYYY-MM' format
cpi_df.index = pd.to_datetime(cpi_df.index)  # Ensure the index is a datetime object
cpi_df.index = cpi_df.index.strftime('%Y-%m')  # Format the index to 'YYYY-MM'

cpi_df

,USD CPI,EUR CPI,GBP CPI,CHF CPI,CAD CPI
2000-02,71.64043,75.10,72.8,93.14778,74.34817
2000-03,72.23110,75.37,73.0,93.11600,74.90124
2000-04,72.27330,75.45,73.3,93.19740,74.66421
2000-05,72.35768,75.54,73.5,93.15159,74.98025
2000-06,72.73740,75.80,73.6,93.49832,75.45431
...,...,...,...,...,...
2024-07,132.70780,126.50,132.9,107.79790,128.07480
2024-08,132.81580,126.67,133.4,107.78250,127.83780
2024-09,133.02890,126.56,133.5,107.46790,127.28470
2024-10,133.18200,126.99,134.3,107.33180,127.83780


In [2]:
# Count of NaN values in each column
nan_columns = cpi_df.isnull().sum()

# Filter columns with NaN count greater than 0
nan_columns = nan_columns[nan_columns > 0]

nan_columns

GBP CPI    1
CAD CPI    1
dtype: int64

In [3]:
# Replace NaN values with the value from the previous row
cpi_df.ffill(inplace=True)

cpi_df

,USD CPI,EUR CPI,GBP CPI,CHF CPI,CAD CPI
2000-02,71.64043,75.10,72.8,93.14778,74.34817
2000-03,72.23110,75.37,73.0,93.11600,74.90124
2000-04,72.27330,75.45,73.3,93.19740,74.66421
2000-05,72.35768,75.54,73.5,93.15159,74.98025
2000-06,72.73740,75.80,73.6,93.49832,75.45431
...,...,...,...,...,...
2024-07,132.70780,126.50,132.9,107.79790,128.07480
2024-08,132.81580,126.67,133.4,107.78250,127.83780
2024-09,133.02890,126.56,133.5,107.46790,127.28470
2024-10,133.18200,126.99,134.3,107.33180,127.83780


### GDP

In [4]:
# Define countries and their GDP tickers
gdp_tickers = {
    "USD GDP": "NGDPRSAXDCUSQ",
    "EUR GDP": "CLVMEURSCAB1GQEA19",
    "GBP GDP": "NGDPRSAXDCGBQ",
    "CHF GDP": "CLVMNACSAB1GQCH",
    "CAD GDP": "NGDPRSAXDCCAQ",
}

# Fetch GDP data for each country
gdp_data = {}
for country, ticker in gdp_tickers.items():
    try:
        gdp_data[country] = fred.get_series(ticker)
    except ValueError as e:
        print(f"Error fetching data for {country} ({ticker}): {e}")

# Convert to DataFrames
gdp_dataframes = {}
for country, data in gdp_data.items():
    gdp_dataframes[country] = pd.DataFrame(data, columns=[country])

# Filter data for Q1 2000 to today
for country, df in gdp_dataframes.items():
    gdp_dataframes[country] = df.loc[start_date:end_date]

# Adjust dates to quarter-end (instead of quarter-start as it's the default)
for country, df in gdp_dataframes.items():
    df.index = pd.to_datetime(df.index)  # Ensure the index is datetime
    df.index = df.index.to_period('Q').to_timestamp('Q')  # Convert to quarter-end

# Merge all DataFrames into a single DataFrame
gdp_df = pd.concat(gdp_dataframes.values(), axis=1)

# Transform the index to 'YYYY-MM' format
gdp_df.index = gdp_df.index.strftime('%Y-%m')

gdp_df

,USD GDP,EUR GDP,GBP GDP,CHF GDP,CAD GDP
2000-06,3532727.00,2128082.5,447825.0,129024.2,383125.8125
2000-09,3536328.00,2142396.0,450688.0,130235.7,387053.0000
2000-12,3557441.25,2152133.6,453447.0,131797.5,387818.5000
2001-03,3545780.00,2175968.0,457680.0,131952.1,390001.5000
2001-06,3567923.50,2177149.0,459620.0,132304.0,391228.0000
...,...,...,...,...,...
2023-09,5695233.50,2805955.2,633956.0,196255.1,588909.5000
2023-12,5740150.00,2807060.8,631825.0,196640.7,589018.5000
2024-03,5763386.50,2815698.2,636222.0,197635.1,591591.8125
2024-06,5805976.50,2820539.9,639095.0,199066.2,594729.3125


In [5]:
# Count of NaN values in each column
nan_columns = gdp_df.isnull().sum()

# Filter columns with NaN count greater than 0
nan_columns = nan_columns[nan_columns > 0]

nan_columns

Series([], dtype: int64)

In [6]:
# Replace NaN values with the value from the previous row
gdp_df.ffill(inplace=True)

gdp_df

,USD GDP,EUR GDP,GBP GDP,CHF GDP,CAD GDP
2000-06,3532727.00,2128082.5,447825.0,129024.2,383125.8125
2000-09,3536328.00,2142396.0,450688.0,130235.7,387053.0000
2000-12,3557441.25,2152133.6,453447.0,131797.5,387818.5000
2001-03,3545780.00,2175968.0,457680.0,131952.1,390001.5000
2001-06,3567923.50,2177149.0,459620.0,132304.0,391228.0000
...,...,...,...,...,...
2023-09,5695233.50,2805955.2,633956.0,196255.1,588909.5000
2023-12,5740150.00,2807060.8,631825.0,196640.7,589018.5000
2024-03,5763386.50,2815698.2,636222.0,197635.1,591591.8125
2024-06,5805976.50,2820539.9,639095.0,199066.2,594729.3125


### Interest Rates (APR)

In [7]:
import requests
from io import StringIO

url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_FINMARK,4.0/CHE+EA19+USA+GBR+CAN.M..PA.....?startPeriod=2000-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'

headers = {
    'User-Agent': 'DataDownloadScript/1.0'
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    interest_rates_df = pd.read_csv(StringIO(response.text))
    print(interest_rates_df.head())
else:
    print(f"Failed to fetch data: {response.status_code} {response.reason}")

  STRUCTURE                            STRUCTURE_ID    STRUCTURE_NAME ACTION  \
0  DATAFLOW  OECD.SDD.STES:DSD_STES@DF_FINMARK(4.0)  Financial market      I   
1  DATAFLOW  OECD.SDD.STES:DSD_STES@DF_FINMARK(4.0)  Financial market      I   
2  DATAFLOW  OECD.SDD.STES:DSD_STES@DF_FINMARK(4.0)  Financial market      I   
3  DATAFLOW  OECD.SDD.STES:DSD_STES@DF_FINMARK(4.0)  Financial market      I   
4  DATAFLOW  OECD.SDD.STES:DSD_STES@DF_FINMARK(4.0)  Financial market      I   

  REF_AREA  Reference area FREQ Frequency of observation MEASURE  \
0      GBR  United Kingdom    M                  Monthly    IRLT   
1      GBR  United Kingdom    M                  Monthly    IRLT   
2      GBR  United Kingdom    M                  Monthly  IRSTCI   
3      GBR  United Kingdom    M                  Monthly  IRSTCI   
4      GBR  United Kingdom    M                  Monthly  IRSTCI   

                                             Measure  ... OBS_VALUE  \
0                           Long-term i

In [8]:
# Filter relevant columns
interest_rates_df = interest_rates_df[['TIME_PERIOD', 'REF_AREA', 'Measure', 'OBS_VALUE']]  # Adjust the columns you want to keep

interest_rates_df

,TIME_PERIOD,REF_AREA,Measure,OBS_VALUE
0,2020-11,GBR,Long-term interest rates,0.3836
1,2020-12,GBR,Long-term interest rates,0.3224
2,2000-03,GBR,"Immediate interest rates, call money, interban...",5.7620
3,2000-04,GBR,"Immediate interest rates, call money, interban...",5.9347
4,2000-05,GBR,"Immediate interest rates, call money, interban...",5.9122
...,...,...,...,...
4479,2020-06,GBR,Long-term interest rates,0.2780
4480,2020-07,GBR,Long-term interest rates,0.2094
4481,2020-08,GBR,Long-term interest rates,0.2717
4482,2020-09,GBR,Long-term interest rates,0.2730


In [9]:
# Set 'TIME_PERIOD' as the index
interest_rates_df.set_index('TIME_PERIOD', inplace=True, drop=True)

interest_rates_df

,REF_AREA,Measure,OBS_VALUE
TIME_PERIOD,,,
2020-11,GBR,Long-term interest rates,0.3836
2020-12,GBR,Long-term interest rates,0.3224
2000-03,GBR,"Immediate interest rates, call money, interban...",5.7620
2000-04,GBR,"Immediate interest rates, call money, interban...",5.9347
2000-05,GBR,"Immediate interest rates, call money, interban...",5.9122
...,...,...,...
2020-06,GBR,Long-term interest rates,0.2780
2020-07,GBR,Long-term interest rates,0.2094
2020-08,GBR,Long-term interest rates,0.2717


In [10]:
# Sort the DataFrame in chronological order
interest_rates_df = interest_rates_df.sort_index()

interest_rates_df

,REF_AREA,Measure,OBS_VALUE
TIME_PERIOD,,,
2000-01,USA,Short-term interest rates,5.950000
2000-01,GBR,Long-term interest rates,5.822700
2000-01,EA19,Long-term interest rates,5.698539
2000-01,GBR,"Immediate interest rates, call money, interban...",5.352700
2000-01,GBR,Short-term interest rates,6.136320
...,...,...,...
2024-11,GBR,Short-term interest rates,4.680000
2024-11,USA,Long-term interest rates,4.360000
2024-11,CHE,"Immediate interest rates, call money, interban...",0.730000


In [11]:
# Filter the DataFrame to keep only rows with 'Immediate interest rates, call money, interbank rate' data
interest_rates_df = interest_rates_df[interest_rates_df['Measure'] == 'Immediate interest rates, call money, interbank rate']
interest_rates_df

,REF_AREA,Measure,OBS_VALUE
TIME_PERIOD,,,
2000-01,GBR,"Immediate interest rates, call money, interban...",5.352700
2000-01,USA,"Immediate interest rates, call money, interban...",5.450000
2000-01,CAN,"Immediate interest rates, call money, interban...",4.770000
2000-01,CHE,"Immediate interest rates, call money, interban...",2.062500
2000-01,EA19,"Immediate interest rates, call money, interban...",3.042900
...,...,...,...
2024-11,CAN,"Immediate interest rates, call money, interban...",3.764880
2024-11,EA19,"Immediate interest rates, call money, interban...",3.164967
2024-11,USA,"Immediate interest rates, call money, interban...",4.640345


In [12]:
# Pivot the DataFrame so that countries become columns
interest_rates_df = interest_rates_df.pivot(columns='REF_AREA', values='OBS_VALUE')

interest_rates_df

REF_AREA,CAN,CHE,EA19,GBR,USA
TIME_PERIOD,,,,,
2000-01,4.770000,2.0625,3.042900,5.3527,5.450000
2000-02,4.971400,1.8000,3.275700,5.8408,5.730000
2000-03,5.248600,2.3500,3.510400,5.7620,5.850000
2000-04,5.258600,2.9500,3.685000,5.9347,6.020000
2000-05,5.752900,2.7000,3.920000,5.9122,6.270000
...,...,...,...,...,...
2024-07,4.707471,0.9300,3.663089,5.2000,5.330000
2024-08,4.512980,1.0800,3.664356,4.9500,5.330000
2024-09,4.297422,0.6400,3.592092,4.9500,5.123104


In [13]:
# Rename columns
interest_rates_df.columns = ['CAD APR', 'CHF APR', 'EUR APR','GBP APR', 'USD APR']
interest_rates_df

,CAD APR,CHF APR,EUR APR,GBP APR,USD APR
TIME_PERIOD,,,,,
2000-01,4.770000,2.0625,3.042900,5.3527,5.450000
2000-02,4.971400,1.8000,3.275700,5.8408,5.730000
2000-03,5.248600,2.3500,3.510400,5.7620,5.850000
2000-04,5.258600,2.9500,3.685000,5.9347,6.020000
2000-05,5.752900,2.7000,3.920000,5.9122,6.270000
...,...,...,...,...,...
2024-07,4.707471,0.9300,3.663089,5.2000,5.330000
2024-08,4.512980,1.0800,3.664356,4.9500,5.330000
2024-09,4.297422,0.6400,3.592092,4.9500,5.123104


In [14]:
# Count of NaN values in each column
nan_columns = interest_rates_df.isnull().sum()

# Filter columns with NaN count greater than 0
nan_columns = nan_columns[nan_columns > 0]

nan_columns

Series([], dtype: int64)

In [15]:
# Replace NaN values with the value from the previous row
interest_rates_df.ffill(inplace=True)

interest_rates_df

,CAD APR,CHF APR,EUR APR,GBP APR,USD APR
TIME_PERIOD,,,,,
2000-01,4.770000,2.0625,3.042900,5.3527,5.450000
2000-02,4.971400,1.8000,3.275700,5.8408,5.730000
2000-03,5.248600,2.3500,3.510400,5.7620,5.850000
2000-04,5.258600,2.9500,3.685000,5.9347,6.020000
2000-05,5.752900,2.7000,3.920000,5.9122,6.270000
...,...,...,...,...,...
2024-07,4.707471,0.9300,3.663089,5.2000,5.330000
2024-08,4.512980,1.0800,3.664356,4.9500,5.330000
2024-09,4.297422,0.6400,3.592092,4.9500,5.123104


### Currency

In [16]:
import yfinance as yf

# Define the tickers (USD in front of all as standard)
tickers = ['USDEUR=X', 'USDGBP=X', 'USDCHF=X', 'USDCAD=X']

# Download the data
fx_df = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close']

# Display the data
fx_df

[*********************100%%**********************]  4 of 4 completed


Ticker,USDCAD=X,USDCHF=X,USDEUR=X,USDGBP=X
Date,,,,
2003-10-01,1.31690,1.338200,NaN,NaN
2003-11-01,1.29800,1.288400,NaN,NaN
2003-12-01,1.29720,1.236600,0.79428,0.559030
2004-01-01,1.32500,1.252900,0.80308,0.548100
2004-02-01,1.33430,1.257400,0.79808,0.533480
...,...,...,...,...
2024-09-01,1.35088,0.840500,0.89527,0.747200
2024-10-01,1.39080,0.866300,0.92088,0.771630
2024-11-01,1.40140,0.880782,0.94510,0.785361


In [17]:
# Transform the index to 'YYYY-MM' format
fx_df.index = fx_df.index.strftime('%Y-%m')

fx_df

Ticker,USDCAD=X,USDCHF=X,USDEUR=X,USDGBP=X
Date,,,,
2003-10,1.31690,1.338200,NaN,NaN
2003-11,1.29800,1.288400,NaN,NaN
2003-12,1.29720,1.236600,0.79428,0.559030
2004-01,1.32500,1.252900,0.80308,0.548100
2004-02,1.33430,1.257400,0.79808,0.533480
...,...,...,...,...
2024-09,1.35088,0.840500,0.89527,0.747200
2024-10,1.39080,0.866300,0.92088,0.771630
2024-11,1.40140,0.880782,0.94510,0.785361


In [18]:
# Rename columns
fx_df.columns = ['CAD', 'CHF', 'EUR', 'GBP']
fx_df

,CAD,CHF,EUR,GBP
Date,,,,
2003-10,1.31690,1.338200,NaN,NaN
2003-11,1.29800,1.288400,NaN,NaN
2003-12,1.29720,1.236600,0.79428,0.559030
2004-01,1.32500,1.252900,0.80308,0.548100
2004-02,1.33430,1.257400,0.79808,0.533480
...,...,...,...,...
2024-09,1.35088,0.840500,0.89527,0.747200
2024-10,1.39080,0.866300,0.92088,0.771630
2024-11,1.40140,0.880782,0.94510,0.785361


In [19]:
fx_df['USD'] = 1
fx_df

,CAD,CHF,EUR,GBP,USD
Date,,,,,
2003-10,1.31690,1.338200,NaN,NaN,1
2003-11,1.29800,1.288400,NaN,NaN,1
2003-12,1.29720,1.236600,0.79428,0.559030,1
2004-01,1.32500,1.252900,0.80308,0.548100,1
2004-02,1.33430,1.257400,0.79808,0.533480,1
...,...,...,...,...,...
2024-09,1.35088,0.840500,0.89527,0.747200,1
2024-10,1.39080,0.866300,0.92088,0.771630,1
2024-11,1.40140,0.880782,0.94510,0.785361,1


In [20]:
# Initialize an empty DataFrame to store the currency ratios
currency_df = pd.DataFrame(index=fx_df.index)

# Get the list of currencies (column names)
currencies = fx_df.columns

# Loop through each pair of currencies
for i in range(len(currencies)):
    for j in range(len(currencies)):
        if i != j:  # Avoid calculating the same currency against itself
            col_name = f"{currencies[j]}/{currencies[i]}"
            currency_df[col_name] = fx_df[currencies[i]] / fx_df[currencies[j]]

# Display the resulting DataFrame
currency_df

,CHF/CAD,EUR/CAD,GBP/CAD,USD/CAD,CAD/CHF,EUR/CHF,GBP/CHF,USD/CHF,CAD/EUR,CHF/EUR,GBP/EUR,USD/EUR,CAD/GBP,CHF/GBP,EUR/GBP,USD/GBP,CAD/USD,CHF/USD,EUR/USD,GBP/USD
Date,,,,,,,,,,,,,,,,,,,,
2003-10,0.984083,NaN,NaN,1.31690,1.016174,NaN,NaN,1.338200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.759359,0.747272,NaN,NaN
2003-11,1.007451,NaN,NaN,1.29800,0.992604,NaN,NaN,1.288400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.770416,0.776156,NaN,NaN
2003-12,1.049005,1.633177,2.320448,1.29720,0.953284,1.556882,2.212046,1.236600,0.612303,0.642310,1.420818,0.79428,0.430951,0.452070,0.703820,0.559030,0.770891,0.808669,1.259002,1.788813
2004-01,1.057547,1.649898,2.417442,1.32500,0.945585,1.560119,2.285897,1.252900,0.606098,0.640977,1.465207,0.80308,0.413660,0.437465,0.682497,0.548100,0.754717,0.798148,1.245206,1.824485
2004-02,1.061158,1.671888,2.501125,1.33430,0.942367,1.575531,2.356977,1.257400,0.598126,0.634707,1.495989,0.79808,0.399820,0.424272,0.668454,0.533480,0.749457,0.795292,1.253007,1.874485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09,1.607234,1.508908,1.807923,1.35088,0.622187,0.938823,1.124866,0.840500,0.662731,1.065164,1.198166,0.89527,0.553121,0.888995,0.834609,0.747200,0.740258,1.189768,1.116981,1.338330
2024-10,1.605448,1.510294,1.802418,1.39080,0.622879,0.940731,1.122688,0.866300,0.662123,1.063004,1.193422,0.92088,0.554810,0.890719,0.837927,0.771630,0.719011,1.154335,1.085918,1.295958
2024-11,1.591086,1.482806,1.784402,1.40140,0.628502,0.931946,1.121500,0.880782,0.674397,1.073024,1.203396,0.94510,0.560412,0.891663,0.830982,0.785361,0.713572,1.135355,1.058089,1.273300


# Merge dfs

In [21]:
import os

# Right=first df, left=second df
# Outer=ensures that all rows from both DataFrames are included (rows with missing values in one DataFrame will have NaN)
merged_dfs = pd.merge(cpi_df, gdp_df, left_index=True, right_index=True, how='inner')
merged_dfs = pd.merge(merged_dfs, interest_rates_df, left_index=True, right_index=True, how='inner')
merged_dfs = pd.merge(merged_dfs, currency_df, left_index=True, right_index=True, how='inner')
merged_dfs

# Define the directory
directory = '04. Exchange Rates Forecast FRED-OECD-Yahoo'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the updated DataFrame to a CSV in the specified directory
output_file = os.path.join(directory, 'Exchange Rates Forecast FRED-OECD-Yahoo_merged_dfs.csv')
merged_dfs.to_csv(output_file)

# Print to confirm save location and show the DataFrame
print(f"File saved to: {output_file}")

File saved to: 04. Exchange Rates Forecast FRED-OECD-Yahoo\Exchange Rates Forecast FRED-OECD-Yahoo_merged_dfs.csv


# Calculate Differences, Percentages and Decimals

In [22]:
countries = ['USD', 'EUR', 'GBP', 'CHF', 'CAD']

# Initialize an empty list to store the results
countries_cpi = []
countries_interest_rates = []
countries_gdp = []

# Loop through each country and append the desired format
for country in countries:
    countries_cpi.append(f"{country} CPI")
    countries_interest_rates.append(f"{country} APR")
    countries_gdp.append(f"{country} GDP")

countries_cpi, countries_interest_rates, countries_gdp

(['USD CPI', 'EUR CPI', 'GBP CPI', 'CHF CPI', 'CAD CPI'],
 ['USD APR', 'EUR APR', 'GBP APR', 'CHF APR', 'CAD APR'],
 ['USD GDP', 'EUR GDP', 'GBP GDP', 'CHF GDP', 'CAD GDP'])

### CPI Differences

In [23]:
cpi_df_2 = merged_dfs[countries_cpi]
cpi_df_2

,USD CPI,EUR CPI,GBP CPI,CHF CPI,CAD CPI
2003-12,77.75813,81.79,77.3,95.78070,81.53806
2004-03,79.06606,82.32,77.3,95.67625,82.09113
2004-06,80.03645,83.05,77.9,96.87312,83.03924
2004-09,80.12083,83.14,77.9,96.22005,82.96023
2004-12,80.28960,83.72,78.6,97.05637,83.27627
...,...,...,...,...,...
2023-09,129.85950,124.40,130.1,106.56950,125.23040
2023-12,129.41940,124.02,130.5,106.46120,125.07240
2024-03,131.77620,125.29,131.6,107.35490,126.25760
2024-06,132.55380,126.55,133.0,107.99090,127.52170


In [24]:
# Calculate 4-quarter accumulated percentage change (year-over-year percentage change for quarterly data)
cpi_4_quarter_pct_change_decimals = cpi_df_2.pct_change(periods=4)

# Exclude rows with NaN values (first 4 quarters will be NaN)
cpi_4_quarter_pct_change_decimals = cpi_4_quarter_pct_change_decimals.dropna()

cpi_4_quarter_pct_change_decimals

,USD CPI,EUR CPI,GBP CPI,CHF CPI,CAD CPI
2004-12,0.032556,0.023597,0.016818,0.013319,0.021318
2005-03,0.031483,0.021866,0.019405,0.014411,0.023099
2005-06,0.025303,0.020470,0.019255,0.006809,0.017127
2005-09,0.046867,0.025980,0.025674,0.013966,0.032381
2005-12,0.034157,0.022695,0.021628,0.010064,0.020873
...,...,...,...,...,...
2023-09,0.036997,0.043099,0.063778,0.016547,0.037982
2023-12,0.033521,0.029041,0.041500,0.017150,0.033965
2024-03,0.034774,0.024113,0.037855,0.010449,0.028977
2024-06,0.029714,0.025111,0.027821,0.013281,0.026717


In [25]:
# Initialize an empty DataFrame for the inflation rate differentials
cpi_difference_decimals = pd.DataFrame(index=cpi_4_quarter_pct_change_decimals.index)

# Calculate the inflation rate differentials
countries = cpi_4_quarter_pct_change_decimals.columns
for i in range(len(countries)):
    for j in range(len(countries)):
        if i != j:  # Avoid calculating differential with itself
            col_name = f"{countries[i]}-{countries[j]}"
            cpi_difference_decimals[col_name] = cpi_4_quarter_pct_change_decimals[countries[i]] - cpi_4_quarter_pct_change_decimals[countries[j]]

cpi_difference_decimals

,USD CPI-EUR CPI,USD CPI-GBP CPI,USD CPI-CHF CPI,USD CPI-CAD CPI,EUR CPI-USD CPI,EUR CPI-GBP CPI,EUR CPI-CHF CPI,EUR CPI-CAD CPI,GBP CPI-USD CPI,GBP CPI-EUR CPI,GBP CPI-CHF CPI,GBP CPI-CAD CPI,CHF CPI-USD CPI,CHF CPI-EUR CPI,CHF CPI-GBP CPI,CHF CPI-CAD CPI,CAD CPI-USD CPI,CAD CPI-EUR CPI,CAD CPI-GBP CPI,CAD CPI-CHF CPI
2004-12,0.008959,0.015738,0.019237,0.011238,-0.008959,0.006779,0.010278,0.002279,-0.015738,-0.006779,0.003499,-0.004500,-0.019237,-0.010278,-0.003499,-0.007999,-0.011238,-0.002279,0.004500,0.007999
2005-03,0.009618,0.012079,0.017072,0.008384,-0.009618,0.002461,0.007455,-0.001233,-0.012079,-0.002461,0.004994,-0.003694,-0.017072,-0.007455,-0.004994,-0.008688,-0.008384,0.001233,0.003694,0.008688
2005-06,0.004833,0.006048,0.018494,0.008176,-0.004833,0.001214,0.013661,0.003343,-0.006048,-0.001214,0.012447,0.002129,-0.018494,-0.013661,-0.012447,-0.010318,-0.008176,-0.003343,-0.002129,0.010318
2005-09,0.020887,0.021193,0.032901,0.014486,-0.020887,0.000306,0.012014,-0.006401,-0.021193,-0.000306,0.011708,-0.006707,-0.032901,-0.012014,-0.011708,-0.018415,-0.014486,0.006401,0.006707,0.018415
2005-12,0.011462,0.012528,0.024093,0.013284,-0.011462,0.001066,0.012631,0.001822,-0.012528,-0.001066,0.011565,0.000756,-0.024093,-0.012631,-0.011565,-0.010809,-0.013284,-0.001822,-0.000756,0.010809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,-0.006102,-0.026781,0.020450,-0.000985,0.006102,-0.020678,0.026552,0.005117,0.026781,0.020678,0.047231,0.025795,-0.020450,-0.026552,-0.047231,-0.021435,0.000985,-0.005117,-0.025795,0.021435
2023-12,0.004480,-0.007979,0.016372,-0.000443,-0.004480,-0.012460,0.011891,-0.004924,0.007979,0.012460,0.024351,0.007536,-0.016372,-0.011891,-0.024351,-0.016815,0.000443,0.004924,-0.007536,0.016815
2024-03,0.010661,-0.003081,0.024325,0.005797,-0.010661,-0.013742,0.013664,-0.004864,0.003081,0.013742,0.027405,0.008878,-0.024325,-0.013664,-0.027405,-0.018527,-0.005797,0.004864,-0.008878,0.018527
2024-06,0.004603,0.001894,0.016434,0.002997,-0.004603,-0.002709,0.011831,-0.001606,-0.001894,0.002709,0.014540,0.001103,-0.016434,-0.011831,-0.014540,-0.013437,-0.002997,0.001606,-0.001103,0.013437


### Interest Rates Differences

In [26]:
interest_rates_df_2 = merged_dfs[countries_interest_rates]
interest_rates_df_2

,USD APR,EUR APR,GBP APR,CHF APR,CAD APR
2003-12,0.980000,2.056700,3.7778,0.09,2.748100
2004-03,1.000000,2.005700,3.9040,0.10,2.248200
2004-06,1.030000,2.027700,4.3628,0.36,2.000500
2004-09,1.610000,2.051800,4.7314,0.54,2.249600
2004-12,2.160000,2.053500,4.7618,0.55,2.499900
...,...,...,...,...,...
2023-09,5.330000,3.712522,5.1854,1.20,4.991785
2023-12,5.330000,3.903442,5.1870,1.60,5.015479
2024-03,5.330000,3.907311,5.1891,1.34,5.002890
2024-06,5.330000,3.786032,5.2000,0.92,4.801320


In [27]:
interest_rates_decimals = interest_rates_df_2 /100
interest_rates_decimals

,USD APR,EUR APR,GBP APR,CHF APR,CAD APR
2003-12,0.009800,0.020567,0.037778,0.0009,0.027481
2004-03,0.010000,0.020057,0.039040,0.0010,0.022482
2004-06,0.010300,0.020277,0.043628,0.0036,0.020005
2004-09,0.016100,0.020518,0.047314,0.0054,0.022496
2004-12,0.021600,0.020535,0.047618,0.0055,0.024999
...,...,...,...,...,...
2023-09,0.053300,0.037125,0.051854,0.0120,0.049918
2023-12,0.053300,0.039034,0.051870,0.0160,0.050155
2024-03,0.053300,0.039073,0.051891,0.0134,0.050029
2024-06,0.053300,0.037860,0.052000,0.0092,0.048013


In [28]:
# Initialize an empty DataFrame for the interest rates differentials
interest_rates_difference_decimals = pd.DataFrame(index=interest_rates_decimals.index)

# Calculate the inflation rate differentials
countries = interest_rates_decimals.columns
for i in range(len(countries)):
    for j in range(len(countries)):
        if i != j:  # Avoid calculating differential with itself
            col_name = f"{countries[i]}-{countries[j]}"
            interest_rates_difference_decimals[col_name] = interest_rates_decimals[countries[i]] - interest_rates_decimals[countries[j]]

interest_rates_difference_decimals

,USD APR-EUR APR,USD APR-GBP APR,USD APR-CHF APR,USD APR-CAD APR,EUR APR-USD APR,EUR APR-GBP APR,EUR APR-CHF APR,EUR APR-CAD APR,GBP APR-USD APR,GBP APR-EUR APR,GBP APR-CHF APR,GBP APR-CAD APR,CHF APR-USD APR,CHF APR-EUR APR,CHF APR-GBP APR,CHF APR-CAD APR,CAD APR-USD APR,CAD APR-EUR APR,CAD APR-GBP APR,CAD APR-CHF APR
2003-12,-0.010767,-0.027978,0.008900,-0.017681,0.010767,-0.017211,0.019667,-0.006914,0.027978,0.017211,0.036878,0.010297,-0.008900,-0.019667,-0.036878,-0.026581,0.017681,0.006914,-0.010297,0.026581
2004-03,-0.010057,-0.029040,0.009000,-0.012482,0.010057,-0.018983,0.019057,-0.002425,0.029040,0.018983,0.038040,0.016558,-0.009000,-0.019057,-0.038040,-0.021482,0.012482,0.002425,-0.016558,0.021482
2004-06,-0.009977,-0.033328,0.006700,-0.009705,0.009977,-0.023351,0.016677,0.000272,0.033328,0.023351,0.040028,0.023623,-0.006700,-0.016677,-0.040028,-0.016405,0.009705,-0.000272,-0.023623,0.016405
2004-09,-0.004418,-0.031214,0.010700,-0.006396,0.004418,-0.026796,0.015118,-0.001978,0.031214,0.026796,0.041914,0.024818,-0.010700,-0.015118,-0.041914,-0.017096,0.006396,0.001978,-0.024818,0.017096
2004-12,0.001065,-0.026018,0.016100,-0.003399,-0.001065,-0.027083,0.015035,-0.004464,0.026018,0.027083,0.042118,0.022619,-0.016100,-0.015035,-0.042118,-0.019499,0.003399,0.004464,-0.022619,0.019499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.016175,0.001446,0.041300,0.003382,-0.016175,-0.014729,0.025125,-0.012793,-0.001446,0.014729,0.039854,0.001936,-0.041300,-0.025125,-0.039854,-0.037918,-0.003382,0.012793,-0.001936,0.037918
2023-12,0.014266,0.001430,0.037300,0.003145,-0.014266,-0.012836,0.023034,-0.011120,-0.001430,0.012836,0.035870,0.001715,-0.037300,-0.023034,-0.035870,-0.034155,-0.003145,0.011120,-0.001715,0.034155
2024-03,0.014227,0.001409,0.039900,0.003271,-0.014227,-0.012818,0.025673,-0.010956,-0.001409,0.012818,0.038491,0.001862,-0.039900,-0.025673,-0.038491,-0.036629,-0.003271,0.010956,-0.001862,0.036629
2024-06,0.015440,0.001300,0.044100,0.005287,-0.015440,-0.014140,0.028660,-0.010153,-0.001300,0.014140,0.042800,0.003987,-0.044100,-0.028660,-0.042800,-0.038813,-0.005287,0.010153,-0.003987,0.038813


### GDP Percentage Change

In [29]:
gdp_df_2 = merged_dfs[countries_gdp]
gdp_df_2

,USD GDP,EUR GDP,GBP GDP,CHF GDP,CAD GDP
2003-12,3790690.00,2232061.1,490059.0,133799.8,413121.0000
2004-03,3812170.00,2244466.9,492014.0,134704.1,416125.0000
2004-06,3841712.50,2258110.1,494824.0,135519.6,421055.5000
2004-09,3878154.75,2263448.2,496642.0,135717.8,426033.8125
2004-12,3917720.00,2272995.0,499020.0,136013.5,429108.5000
...,...,...,...,...,...
2023-09,5695233.50,2805955.2,633956.0,196255.1,588909.5000
2023-12,5740150.00,2807060.8,631825.0,196640.7,589018.5000
2024-03,5763386.50,2815698.2,636222.0,197635.1,591591.8125
2024-06,5805976.50,2820539.9,639095.0,199066.2,594729.3125


In [30]:
# Calculate 4-quarter accumulated percentage change (year-over-year percentage change for quarterly data)
gdp_pct_change_decimals = gdp_df_2.pct_change(periods=4)

# Exclude rows with NaN values (first 4 quarters will be NaN)
gdp_pct_change_decimals = gdp_pct_change_decimals.dropna()

gdp_pct_change_decimals

,USD GDP,EUR GDP,GBP GDP,CHF GDP,CAD GDP
2004-12,0.033511,0.018339,0.018286,0.016545,0.038699
2005-03,0.039088,0.015440,0.021723,0.016100,0.034812
2005-06,0.036177,0.015508,0.024417,0.024199,0.030086
2005-09,0.034486,0.020781,0.029176,0.030317,0.030387
2005-12,0.029727,0.023454,0.033896,0.039256,0.033164
...,...,...,...,...,...
2023-09,0.032363,-0.000370,0.003131,0.003026,0.007310
2023-12,0.031962,0.000869,-0.003500,0.003807,0.009671
2024-03,0.029018,0.004265,0.002537,0.003051,0.005556
2024-06,0.030368,0.005255,0.007045,0.015755,0.009140


###  Currency Percentage Change

In [31]:
currency_df_2 = merged_dfs[currency_df.columns]
currency_df_2

,CHF/CAD,EUR/CAD,GBP/CAD,USD/CAD,CAD/CHF,EUR/CHF,GBP/CHF,USD/CHF,CAD/EUR,CHF/EUR,GBP/EUR,USD/EUR,CAD/GBP,CHF/GBP,EUR/GBP,USD/GBP,CAD/USD,CHF/USD,EUR/USD,GBP/USD
2003-12,1.049005,1.633177,2.320448,1.29720,0.953284,1.556882,2.212046,1.236600,0.612303,0.642310,1.420818,0.79428,0.430951,0.452070,0.703820,0.559030,0.770891,0.808669,1.259002,1.788813
2004-03,1.034439,1.612510,2.414098,1.30960,0.966707,1.558825,2.333727,1.266000,0.620151,0.641509,1.497106,0.81215,0.414233,0.428499,0.667955,0.542480,0.763592,0.789889,1.231300,1.843386
2004-06,1.066299,1.624490,2.426873,1.33330,0.937823,1.523485,2.275979,1.250400,0.615578,0.656390,1.493930,0.82075,0.412053,0.439371,0.669376,0.549390,0.750019,0.799744,1.218398,1.820201
2004-09,1.012846,1.568542,2.286859,1.26150,0.987317,1.548648,2.257854,1.245500,0.637535,0.645725,1.457952,0.80425,0.437281,0.442898,0.685894,0.551630,0.792707,0.802890,1.243394,1.812809
2004-12,1.053135,1.629295,2.305861,1.20110,0.949546,1.547091,2.189522,1.140500,0.613762,0.646374,1.415251,0.73719,0.433677,0.456721,0.706589,0.520890,0.832570,0.876808,1.356502,1.919791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,1.484703,1.438645,1.658143,1.35880,0.673535,0.968978,1.116818,0.915200,0.695099,1.032015,1.152573,0.94450,0.603084,0.895401,0.867624,0.819471,0.735943,1.092657,1.058761,1.220299
2023-12,1.577453,1.466114,1.689583,1.32610,0.633933,0.929418,1.071083,0.840659,0.682075,1.075942,1.152423,0.90450,0.591862,0.933634,0.867737,0.784868,0.754091,1.189543,1.105583,1.274100
2024-03,1.501216,1.462411,1.709569,1.35390,0.666127,0.974151,1.138790,0.901869,0.683802,1.026535,1.169007,0.92580,0.584943,0.878125,0.855427,0.791954,0.738607,1.108808,1.080147,1.262700
2024-06,1.516286,1.467996,1.732038,1.36920,0.659506,0.968153,1.142290,0.902996,0.681201,1.032895,1.179865,0.93270,0.577355,0.875435,0.847554,0.790514,0.730353,1.107425,1.072156,1.265000


In [32]:
# Calculate percentage change
currency_1period_pct_change_decimals = currency_df_2.pct_change(periods=1)

# Exclude rows with NaN values
currency_1period_pct_change_decimals = currency_1period_pct_change_decimals.dropna()

# Rename columns with a specific label
currency_1period_pct_change_decimals.columns = [f"{col} 1 Period" for col in currency_df_2.columns]

currency_1period_pct_change_decimals

,CHF/CAD 1 Period,EUR/CAD 1 Period,GBP/CAD 1 Period,USD/CAD 1 Period,CAD/CHF 1 Period,EUR/CHF 1 Period,GBP/CHF 1 Period,USD/CHF 1 Period,CAD/EUR 1 Period,CHF/EUR 1 Period,GBP/EUR 1 Period,USD/EUR 1 Period,CAD/GBP 1 Period,CHF/GBP 1 Period,EUR/GBP 1 Period,USD/GBP 1 Period,CAD/USD 1 Period,CHF/USD 1 Period,EUR/USD 1 Period,GBP/USD 1 Period
2004-03,-0.013886,-0.012655,0.040359,0.009559,0.014081,0.001248,0.055008,0.023775,0.012817,-0.001247,0.053693,0.022498,-0.038793,-0.052140,-0.050957,-0.029605,-0.009469,-0.023223,-0.022003,0.030508
2004-06,0.030799,0.007429,0.005292,0.018097,-0.029879,-0.022671,-0.024745,-0.012322,-0.007374,0.023197,-0.002122,0.010589,-0.005264,0.025373,0.002126,0.012738,-0.017775,0.012476,-0.010478,-0.012578
2004-09,-0.050129,-0.034440,-0.057693,-0.053851,0.052775,0.016517,-0.007964,-0.003919,0.035669,-0.016249,-0.024083,-0.020104,0.061226,0.008027,0.024677,0.004077,0.056916,0.003934,0.020516,-0.004061
2004-12,0.039777,0.038732,0.008309,-0.047880,-0.038256,-0.001005,-0.030264,-0.084304,-0.037288,0.001006,-0.029288,-0.083382,-0.008241,0.031209,0.030172,-0.055726,0.050287,0.092065,0.090967,0.059014
2005-03,-0.039496,-0.037323,-0.008812,0.007077,0.041120,0.002262,0.031945,0.048488,0.038770,-0.002257,0.029616,0.046121,0.008891,-0.030956,-0.028764,0.016030,-0.007027,-0.046245,-0.044088,-0.015777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.007514,-0.000951,-0.007955,0.025509,-0.007458,-0.008401,-0.015353,0.017862,0.000952,0.008473,-0.007011,0.026485,0.008019,0.015593,0.007060,0.033733,-0.024875,-0.017548,-0.025802,-0.032632
2023-12,0.062470,0.019094,0.018961,-0.024065,-0.058797,-0.040826,-0.040951,-0.081448,-0.018736,0.042564,-0.000130,-0.042350,-0.018608,0.042699,0.000130,-0.042226,0.024659,0.088670,0.044223,0.044088
2024-03,-0.048329,-0.002526,0.011829,0.020964,0.050784,0.048130,0.063213,0.072812,0.002532,-0.045919,0.014391,0.023549,-0.011690,-0.059455,-0.014187,0.009028,-0.020533,-0.067870,-0.023007,-0.008947
2024-06,0.010039,0.003819,0.013143,0.011301,-0.009939,-0.006157,0.003073,0.001250,-0.003805,0.006196,0.009288,0.007453,-0.012972,-0.003064,-0.009203,-0.001818,-0.011174,-0.001248,-0.007398,0.001822


In [33]:
# Calculate percentage change
currency_4period_pct_change_decimals = currency_df_2.pct_change(periods=4)

# Exclude rows with NaN values
currency_4period_pct_change_decimals = currency_4period_pct_change_decimals.dropna()

# Rename columns only for those not already renamed
currency_4period_pct_change_decimals.columns = [
    f"{col} 4 Periods" if "Period" not in col else col for col in currency_4period_pct_change_decimals.columns
]

currency_4period_pct_change_decimals

,CHF/CAD 4 Periods,EUR/CAD 4 Periods,GBP/CAD 4 Periods,USD/CAD 4 Periods,CAD/CHF 4 Periods,EUR/CHF 4 Periods,GBP/CHF 4 Periods,USD/CHF 4 Periods,CAD/EUR 4 Periods,CHF/EUR 4 Periods,GBP/EUR 4 Periods,USD/EUR 4 Periods,CAD/GBP 4 Periods,CHF/GBP 4 Periods,EUR/GBP 4 Periods,USD/GBP 4 Periods,CAD/USD 4 Periods,CHF/USD 4 Periods,EUR/USD 4 Periods,GBP/USD 4 Periods
2004-12,0.003936,-0.002377,-0.006286,-0.074083,-0.003921,-0.006289,-0.010183,-0.077713,0.002383,0.006329,-0.003918,-0.071876,0.006326,0.010287,0.003934,-0.068225,0.080010,0.084261,0.077443,0.073221
2005-03,-0.022136,-0.027302,-0.053253,-0.076359,0.022637,-0.005283,-0.031821,-0.055450,0.028068,0.005311,-0.026679,-0.050434,0.056248,0.032866,0.027410,-0.024406,0.082672,0.058706,0.053113,0.025017
2005-06,-0.104861,-0.088460,-0.096548,-0.081452,0.117145,0.018323,0.009287,0.026152,0.097045,-0.017993,-0.008873,0.007688,0.106866,-0.009202,0.008953,0.016709,0.088675,-0.025485,-0.007629,-0.016435
2005-09,-0.112445,-0.108330,-0.102752,-0.078240,0.126691,0.004637,0.010921,0.038539,0.121492,-0.004615,0.006256,0.033746,0.114519,-0.010803,-0.006217,0.027319,0.084881,-0.037109,-0.032644,-0.026592
2005-12,-0.160473,-0.155732,-0.133332,-0.033053,0.191147,0.005647,0.032329,0.151776,0.184458,-0.005615,0.026533,0.145309,0.153844,-0.031317,-0.025847,0.115706,0.034183,-0.131775,-0.126873,-0.103706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.058526,0.070307,0.085623,-0.006326,-0.055290,0.011129,0.025599,-0.061266,-0.065688,-0.011007,0.014310,-0.071598,-0.078870,-0.024960,-0.014108,-0.084697,0.006366,0.065264,0.077120,0.092534
2023-12,0.074891,0.007345,0.029387,-0.022483,-0.069673,-0.062840,-0.042334,-0.090590,-0.007292,0.067053,0.021881,-0.029611,-0.028548,0.044205,-0.021412,-0.050389,0.023000,0.099613,0.030514,0.053062
2024-03,0.014044,-0.008210,0.020641,0.001265,-0.013850,-0.021946,0.006506,-0.012603,0.008277,0.022438,0.029090,0.009553,-0.020224,-0.006464,-0.028267,-0.018985,-0.001263,0.012764,-0.009462,0.019352
2024-06,0.028946,0.019432,0.036255,0.033358,-0.028131,-0.009246,0.007104,0.004289,-0.019062,0.009333,0.016503,0.013661,-0.034987,-0.007054,-0.016235,-0.002795,-0.032282,-0.004270,-0.013477,0.002803


In [34]:
# Calculate percentage change
currency_8period_pct_change_decimals = currency_df_2.pct_change(periods=8)

# Exclude rows with NaN values
currency_8period_pct_change_decimals = currency_8period_pct_change_decimals.dropna()

# Rename columns only for those not already renamed
currency_8period_pct_change_decimals.columns = [
    f"{col} 8 Periods" if "Period" not in col else col for col in currency_8period_pct_change_decimals.columns
]

currency_8period_pct_change_decimals

,CHF/CAD 8 Periods,EUR/CAD 8 Periods,GBP/CAD 8 Periods,USD/CAD 8 Periods,CAD/CHF 8 Periods,EUR/CHF 8 Periods,GBP/CHF 8 Periods,USD/CHF 8 Periods,CAD/EUR 8 Periods,CHF/EUR 8 Periods,GBP/EUR 8 Periods,USD/EUR 8 Periods,CAD/GBP 8 Periods,CHF/GBP 8 Periods,EUR/GBP 8 Periods,USD/GBP 8 Periods,CAD/USD 8 Periods,CHF/USD 8 Periods,EUR/USD 8 Periods,GBP/USD 8 Periods
2005-12,-0.157168,-0.157739,-0.138780,-0.104687,0.186476,-0.000678,0.021817,0.062267,0.187281,0.000678,0.022510,0.062988,0.161143,-0.021351,-0.022015,0.039586,0.116928,-0.058617,-0.059255,-0.038079
2006-03,-0.134759,-0.122914,-0.159685,-0.108583,0.155747,0.013690,-0.028809,0.030253,0.140139,-0.013505,-0.041924,0.016339,0.190030,0.029663,0.043759,0.060813,0.121809,-0.029364,-0.016077,-0.057327
2006-06,-0.145293,-0.122851,-0.150722,-0.163954,0.169992,0.026257,-0.006351,-0.021833,0.140058,-0.025585,-0.031774,-0.046860,0.177470,0.006392,0.032816,-0.015581,0.196107,0.022320,0.049163,0.015828
2006-09,-0.119574,-0.096322,-0.085857,-0.114625,0.135813,0.026410,0.038296,0.005620,0.106588,-0.025731,0.011580,-0.020255,0.093921,-0.036883,-0.011447,-0.031470,0.129466,-0.005589,0.020674,0.032493
2006-12,-0.091901,-0.055813,-0.009631,-0.029556,0.101202,0.039741,0.090596,0.068654,0.059112,-0.038222,0.048912,0.027808,0.009724,-0.083070,-0.046631,-0.020119,0.030456,-0.064244,-0.027056,0.020532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.087585,-0.027710,-0.031673,0.065408,-0.080531,-0.106009,-0.109654,-0.020391,0.028499,0.118580,-0.004077,0.095771,0.032709,0.123158,0.004093,0.100256,-0.061392,0.020815,-0.087401,-0.091121
2023-12,0.130943,0.015802,-0.017848,0.040535,-0.115782,-0.101810,-0.131563,-0.079940,-0.015556,0.113350,-0.033126,0.024349,0.018172,0.151494,0.034261,0.059444,-0.038956,0.086885,-0.023770,-0.056109
2024-03,0.110034,0.049637,0.042398,0.084656,-0.099126,-0.054410,-0.060931,-0.022862,-0.047290,0.057540,-0.006896,0.033363,-0.040674,0.064884,0.006944,0.040539,-0.078049,0.023397,-0.032286,-0.038959
2024-06,0.123012,0.090745,0.108166,0.062771,-0.109537,-0.028732,-0.013219,-0.053642,-0.083195,0.029582,0.015972,-0.025646,-0.097608,0.013396,-0.015721,-0.040964,-0.059064,0.056682,0.026321,0.042714


# Merge Final df

In [35]:
from functools import reduce

# List of DataFrames to merge
dfs = [
    currency_1period_pct_change_decimals,
    currency_4period_pct_change_decimals,
    currency_8period_pct_change_decimals,
    interest_rates_difference_decimals,
    cpi_difference_decimals,
    gdp_pct_change_decimals
]

# Use reduce to iteratively merge the DataFrames using an outer join
final_df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), dfs)

# Save the updated DataFrame to a CSV in the specified directory
output_file = os.path.join(directory, 'Exchange Rates Forecast FRED-OECD-Yahoo_final_df.csv')
final_df.to_csv(output_file)

# Print to confirm save location and show the DataFrame
print(f"File saved to: {output_file}")
final_df

File saved to: 04. Exchange Rates Forecast FRED-OECD-Yahoo\Exchange Rates Forecast FRED-OECD-Yahoo_final_df.csv


,CHF/CAD 1 Period,EUR/CAD 1 Period,GBP/CAD 1 Period,USD/CAD 1 Period,CAD/CHF 1 Period,EUR/CHF 1 Period,GBP/CHF 1 Period,USD/CHF 1 Period,CAD/EUR 1 Period,CHF/EUR 1 Period,...,CHF CPI-CAD CPI,CAD CPI-USD CPI,CAD CPI-EUR CPI,CAD CPI-GBP CPI,CAD CPI-CHF CPI,USD GDP,EUR GDP,GBP GDP,CHF GDP,CAD GDP
2003-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03,-0.013886,-0.012655,0.040359,0.009559,0.014081,0.001248,0.055008,0.023775,0.012817,-0.001247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-06,0.030799,0.007429,0.005292,0.018097,-0.029879,-0.022671,-0.024745,-0.012322,-0.007374,0.023197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-09,-0.050129,-0.034440,-0.057693,-0.053851,0.052775,0.016517,-0.007964,-0.003919,0.035669,-0.016249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-12,0.039777,0.038732,0.008309,-0.047880,-0.038256,-0.001005,-0.030264,-0.084304,-0.037288,0.001006,...,-0.007999,-0.011238,-0.002279,0.004500,0.007999,0.033511,0.018339,0.018286,0.016545,0.038699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,0.007514,-0.000951,-0.007955,0.025509,-0.007458,-0.008401,-0.015353,0.017862,0.000952,0.008473,...,-0.021435,0.000985,-0.005117,-0.025795,0.021435,0.032363,-0.000370,0.003131,0.003026,0.007310
2023-12,0.062470,0.019094,0.018961,-0.024065,-0.058797,-0.040826,-0.040951,-0.081448,-0.018736,0.042564,...,-0.016815,0.000443,0.004924,-0.007536,0.016815,0.031962,0.000869,-0.003500,0.003807,0.009671
2024-03,-0.048329,-0.002526,0.011829,0.020964,0.050784,0.048130,0.063213,0.072812,0.002532,-0.045919,...,-0.018527,-0.005797,0.004864,-0.008878,0.018527,0.029018,0.004265,0.002537,0.003051,0.005556
2024-06,0.010039,0.003819,0.013143,0.011301,-0.009939,-0.006157,0.003073,0.001250,-0.003805,0.006196,...,-0.013437,-0.002997,0.001606,-0.001103,0.013437,0.030368,0.005255,0.007045,0.015755,0.009140


# Regression

### 1 Period

In [36]:
import statsmodels.api as sm

# Filter columns ending with '1 Period'
dependent_vars_1_period = [col for col in final_df.columns if col.endswith('1 Period')]

# Prepare a list to store regression results
regression_results_1_period = []

# Loop through dependent variables dynamically
for y_col in dependent_vars_1_period:
    # Extract currency pair identifiers
    base, quote = y_col.split('/')[0], y_col.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    if apr_col in final_df.columns and cpi_col in final_df.columns and gdp_base_col in final_df.columns and gdp_quote_col in final_df.columns:
        # Prepare data for regression
        X = final_df[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]].dropna()
        Y = final_df[y_col].dropna()
        
        # Align indexes
        aligned_data = pd.concat([X, Y], axis=1).dropna()
        X = aligned_data[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]]
        Y = aligned_data[y_col]
        
        # Add constant to X
        X = sm.add_constant(X)
        
        # Run regression
        model = sm.OLS(Y, X).fit()
        intercept = model.params['const']
        apr_beta = model.params[apr_col]
        cpi_beta = model.params[cpi_col]
        gdp_base_beta = model.params[gdp_base_col]
        gdp_quote_beta = model.params[gdp_quote_col]
        r_squared = model.rsquared
        
        # Append the results to the list
        regression_results_1_period.append({
            'Currency Pair': y_col,
            'Intercept': intercept,
            'APR Beta': apr_beta,
            'CPI Beta': cpi_beta,
            'GDP Base Beta': gdp_base_beta,
            'GDP Quote Beta': gdp_quote_beta,
            'R-squared': r_squared
        })

# Convert the list of results to a DataFrame
regression_results_1_period_df = pd.DataFrame(regression_results_1_period)

# Set the 'Currency Pair' column as the index
regression_results_1_period_df.set_index('Currency Pair', inplace=True)

# Display the DataFrame
regression_results_1_period_df

,Intercept,APR Beta,CPI Beta,GDP Base Beta,GDP Quote Beta,R-squared
Currency Pair,,,,,,
CHF/CAD 1 Period,0.009519,-0.495123,0.861246,-0.055333,0.205876,0.038472
EUR/CAD 1 Period,0.001079,-0.222090,0.758755,0.242428,-0.285346,0.035006
GBP/CAD 1 Period,-0.006687,-0.920231,0.355136,-0.083672,0.221097,0.060921
USD/CAD 1 Period,-0.002104,-1.755043,-0.594246,-0.032914,0.269545,0.115359
CAD/CHF 1 Period,-0.003407,-0.546759,0.704964,-0.183770,0.023555,0.038017
EUR/CHF 1 Period,0.002886,-0.109994,-0.057858,0.251309,-0.493176,0.034699
GBP/CHF 1 Period,-0.006156,0.075723,0.192134,0.102507,-0.424396,0.021572
USD/CHF 1 Period,-0.013015,-0.104901,0.675895,0.060568,-0.212502,0.030273
CAD/EUR 1 Period,0.001735,-0.352840,0.695480,0.285381,-0.237535,0.034729


In [37]:
# Create a DataFrame to store the weighted results
currency_forecast_1_period_df = pd.DataFrame(index=final_df.index)

# Loop through the regression results
for currency_pair, params in regression_results_1_period_df.iterrows():
    # Extract the base and quote currencies
    base, quote = currency_pair.split('/')[0], currency_pair.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    # Check if all necessary columns exist in final_df
    if all(col in final_df.columns for col in [apr_col, cpi_col, gdp_base_col, gdp_quote_col]):
        # Get the most recent data for the explanatory variables
        most_recent_data = final_df.iloc[-1]
        
        # Calculate the weighted result for this currency pair
        weighted_result = (
            params['Intercept'] * 1 +  # Intercept term
            params['APR Beta'] * most_recent_data[apr_col] +
            params['CPI Beta'] * most_recent_data[cpi_col] +
            params['GDP Base Beta'] * most_recent_data[gdp_base_col] +
            params['GDP Quote Beta'] * most_recent_data[gdp_quote_col]
        )
        
        # Store the weighted result in the DataFrame
        currency_forecast_1_period_df.loc[most_recent_data.name, currency_pair] = weighted_result
    else:
        print(f"Missing one or more columns for {currency_pair}. Skipping.")

currency_forecast_1_period_df = currency_forecast_1_period_df.iloc[-1]
currency_forecast_1_period_df

CHF/CAD 1 Period    0.022203
EUR/CAD 1 Period    0.002232
GBP/CAD 1 Period   -0.007403
USD/CAD 1 Period   -0.019016
CAD/CHF 1 Period   -0.019541
EUR/CHF 1 Period   -0.007527
GBP/CHF 1 Period   -0.006246
USD/CHF 1 Period   -0.009182
CAD/EUR 1 Period   -0.000233
CHF/EUR 1 Period    0.009168
GBP/EUR 1 Period    0.001951
USD/EUR 1 Period    0.002751
CAD/GBP 1 Period    0.009394
CHF/GBP 1 Period    0.008503
EUR/GBP 1 Period   -0.001564
USD/GBP 1 Period   -0.003942
CAD/USD 1 Period    0.019455
CHF/USD 1 Period    0.010004
EUR/USD 1 Period   -0.001461
GBP/USD 1 Period    0.004951
Name: 2024-09, dtype: float64

In [38]:
# Create a new DataFrame for the updated values
currency_forecast_1_period_df_2 = pd.DataFrame(index=currency_forecast_1_period_df.index)

# Loop through each currency pair in the forecast DataFrame
for currency_pair in currency_forecast_1_period_df.index:
    # Extract the base/quote from the first 7 characters
    base_quote = currency_pair[:7]
    
    # Ensure the base/quote exists in merged_dfs
    if base_quote in merged_dfs.columns:
        # Get the last value of the base/quote from merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]
        
        # Get the weighted result for the currency pair from currency_forecast_1_period_df
        weighted_result = currency_forecast_1_period_df.loc[currency_pair]
        
        # Calculate the updated value
        updated_value = (last_value * weighted_result) + last_value
        
        # Add the updated value to the new DataFrame
        currency_forecast_1_period_df_2.loc[currency_pair, 'Rates Forecast'] = updated_value
    else:
        print(f"Base/Quote {base_quote} does not exist in merged_dfs. Skipping.")

# Display the final DataFrame
currency_forecast_1_period_df_2

,Rates Forecast
CHF/CAD 1 Period,1.642919
EUR/CAD 1 Period,1.512275
GBP/CAD 1 Period,1.794538
USD/CAD 1 Period,1.325191
CAD/CHF 1 Period,0.610029
EUR/CHF 1 Period,0.931756
GBP/CHF 1 Period,1.117841
USD/CHF 1 Period,0.832782
CAD/EUR 1 Period,0.662577
CHF/EUR 1 Period,1.074929


In [39]:
# Add a new column for percentage changes to the existing DataFrame
currency_forecast_1_period_df_2['Percentage Change'] = None

# Loop through each currency pair
for currency_pair in currency_forecast_1_period_df_2.index:
    base_quote = currency_pair[:7]  # Extract base/quote (first 7 characters of the index)
    
    if base_quote in merged_dfs.columns:  # Check if the column exists in merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]  # Get the last value from merged_dfs
        rates_forecast = currency_forecast_1_period_df_2.loc[currency_pair, 'Rates Forecast']  # Get the forecast value
        
        # Calculate the percentage change
        percentage_change = ((rates_forecast - last_value) / last_value) * 100
        
        # Store the percentage change in the existing DataFrame
        currency_forecast_1_period_df_2.at[currency_pair, 'Percentage Change'] = percentage_change
    else:
        print(f"Column {base_quote} does not exist in merged_dfs. Skipping.")

# Display the updated DataFrame
currency_forecast_1_period_df_2

,Rates Forecast,Percentage Change
CHF/CAD 1 Period,1.642919,2.220271
EUR/CAD 1 Period,1.512275,0.223163
GBP/CAD 1 Period,1.794538,-0.740327
USD/CAD 1 Period,1.325191,-1.901648
CAD/CHF 1 Period,0.610029,-1.954127
EUR/CHF 1 Period,0.931756,-0.752716
GBP/CHF 1 Period,1.117841,-0.624568
USD/CHF 1 Period,0.832782,-0.918242
CAD/EUR 1 Period,0.662577,-0.023268
CHF/EUR 1 Period,1.074929,0.916764


In [40]:
# Right=first df, left=second df
# Outer=ensures that all rows from both DataFrames are included (rows with missing values in one DataFrame will have NaN)
currency_forecast_1_period_df_2 = pd.merge(
    currency_forecast_1_period_df_2,
    regression_results_1_period_df[['R-squared']],  # Select only the 'R-squared' column
    left_index=True,
    right_index=True,
    how='inner'  # Use 'inner' or any other join type you prefer
)

currency_forecast_1_period_df_2

,Rates Forecast,Percentage Change,R-squared
CHF/CAD 1 Period,1.642919,2.220271,0.038472
EUR/CAD 1 Period,1.512275,0.223163,0.035006
GBP/CAD 1 Period,1.794538,-0.740327,0.060921
USD/CAD 1 Period,1.325191,-1.901648,0.115359
CAD/CHF 1 Period,0.610029,-1.954127,0.038017
EUR/CHF 1 Period,0.931756,-0.752716,0.034699
GBP/CHF 1 Period,1.117841,-0.624568,0.021572
USD/CHF 1 Period,0.832782,-0.918242,0.030273
CAD/EUR 1 Period,0.662577,-0.023268,0.034729
CHF/EUR 1 Period,1.074929,0.916764,0.030401


In [41]:
# Update the index to only include the first 7 characters
currency_forecast_1_period_df_2.index = currency_forecast_1_period_df_2.index.str[:7]

# Rename the column
currency_forecast_1_period_df_2.columns = ['Forecast (3M)', 'Change (3M)', 'R-squared (3M)']

# Change numbers format for column 'Change (4M)' to include %
currency_forecast_1_period_df_2['Change (3M)'] = currency_forecast_1_period_df_2['Change (3M)'].apply(lambda x: f"{x:.2f}%")

# Display the updated DataFrame
currency_forecast_1_period_df_2

,Forecast (3M),Change (3M),R-squared (3M)
CHF/CAD,1.642919,2.22%,0.038472
EUR/CAD,1.512275,0.22%,0.035006
GBP/CAD,1.794538,-0.74%,0.060921
USD/CAD,1.325191,-1.90%,0.115359
CAD/CHF,0.610029,-1.95%,0.038017
EUR/CHF,0.931756,-0.75%,0.034699
GBP/CHF,1.117841,-0.62%,0.021572
USD/CHF,0.832782,-0.92%,0.030273
CAD/EUR,0.662577,-0.02%,0.034729
CHF/EUR,1.074929,0.92%,0.030401


### 4 Periods

In [42]:
# Filter columns ending with '4 Period'
dependent_vars_4_periods = [col for col in final_df.columns if col.endswith('4 Periods')]

# Prepare a list to store regression results
regression_results_4_periods = []

# Loop through dependent variables dynamically
for y_col in dependent_vars_4_periods:
    # Extract currency pair identifiers
    base, quote = y_col.split('/')[0], y_col.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    if apr_col in final_df.columns and cpi_col in final_df.columns and gdp_base_col in final_df.columns and gdp_quote_col in final_df.columns:
        # Prepare data for regression
        X = final_df[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]].dropna()
        Y = final_df[y_col].dropna()
        
        # Align indexes
        aligned_data = pd.concat([X, Y], axis=1).dropna()
        X = aligned_data[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]]
        Y = aligned_data[y_col]
        
        # Add constant to X
        X = sm.add_constant(X)
        
        # Run regression
        model = sm.OLS(Y, X).fit()
        intercept = model.params['const']
        apr_beta = model.params[apr_col]
        cpi_beta = model.params[cpi_col]
        gdp_base_beta = model.params[gdp_base_col]
        gdp_quote_beta = model.params[gdp_quote_col]
        r_squared = model.rsquared
        
        # Append the results to the list
        regression_results_4_periods.append({
            'Currency Pair': y_col,
            'Intercept': intercept,
            'APR Beta': apr_beta,
            'CPI Beta': cpi_beta,
            'GDP Base Beta': gdp_base_beta,
            'GDP Quote Beta': gdp_quote_beta,
            'R-squared': r_squared
        })

# Convert the list of results to a DataFrame
regression_results_4_periods_df = pd.DataFrame(regression_results_4_periods)

# Set the 'Currency Pair' column as the index
regression_results_4_periods_df.set_index('Currency Pair', inplace=True)

# Display the DataFrame
regression_results_4_periods_df

,Intercept,APR Beta,CPI Beta,GDP Base Beta,GDP Quote Beta,R-squared
Currency Pair,,,,,,
CHF/CAD 4 Periods,0.037205,-0.089397,-0.526053,-0.938303,-0.300143,0.110129
EUR/CAD 4 Periods,0.021132,0.732200,-0.316848,0.533021,-1.280487,0.091633
GBP/CAD 4 Periods,0.000266,-2.073870,-0.508072,0.426068,-0.859652,0.067970
USD/CAD 4 Periods,0.031827,-1.931667,-5.219778,-0.414800,0.070604,0.406512
CAD/CHF 4 Periods,-0.024693,-0.256813,-0.667148,0.232916,0.986925,0.117479
EUR/CHF 4 Periods,-0.014491,-0.006765,-1.144693,-0.178262,0.624611,0.106381
GBP/CHF 4 Periods,-0.033267,0.289927,-0.616637,0.304563,0.288222,0.088170
USD/CHF 4 Periods,-0.031001,0.503346,0.501859,0.860548,-1.085249,0.064725
CAD/EUR 4 Periods,-0.013931,0.452903,-0.370652,1.368993,-0.574253,0.089247


In [43]:
# Create a DataFrame to store the weighted results
currency_forecast_4_periods_df = pd.DataFrame(index=final_df.index)

# Loop through the regression results
for currency_pair, params in regression_results_4_periods_df.iterrows():
    # Extract the base and quote currencies
    base, quote = currency_pair.split('/')[0], currency_pair.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    # Check if all necessary columns exist in final_df
    if all(col in final_df.columns for col in [apr_col, cpi_col, gdp_base_col, gdp_quote_col]):
        # Get the most recent data for the explanatory variables
        most_recent_data = final_df.iloc[-1]
        
        # Calculate the weighted result for this currency pair
        weighted_result = (
            params['Intercept'] * 1 +  # Intercept term
            params['APR Beta'] * most_recent_data[apr_col] +
            params['CPI Beta'] * most_recent_data[cpi_col] +
            params['GDP Base Beta'] * most_recent_data[gdp_base_col] +
            params['GDP Quote Beta'] * most_recent_data[gdp_quote_col]
        )
        
        # Store the weighted result in the DataFrame
        currency_forecast_4_periods_df.loc[most_recent_data.name, currency_pair] = weighted_result
    else:
        print(f"Missing one or more columns for {currency_pair}. Skipping.")

currency_forecast_4_periods_df = currency_forecast_4_periods_df.iloc[-1]
currency_forecast_4_periods_df

CHF/CAD 4 Periods    0.024001
EUR/CAD 4 Periods    0.005380
GBP/CAD 4 Periods   -0.024379
USD/CAD 4 Periods   -0.036074
CAD/CHF 4 Periods   -0.018651
EUR/CHF 4 Periods   -0.015193
GBP/CHF 4 Periods   -0.023531
USD/CHF 4 Periods    0.002674
CAD/EUR 4 Periods    0.000573
CHF/EUR 4 Periods    0.017304
GBP/EUR 4 Periods   -0.013809
USD/EUR 4 Periods    0.011447
CAD/GBP 4 Periods    0.032023
CHF/GBP 4 Periods    0.028648
EUR/GBP 4 Periods    0.015904
USD/GBP 4 Periods   -0.007363
CAD/USD 4 Periods    0.039001
CHF/USD 4 Periods   -0.002000
EUR/USD 4 Periods   -0.006562
GBP/USD 4 Periods    0.011177
Name: 2024-09, dtype: float64

In [44]:
# Create a new DataFrame for the updated values
currency_forecast_4_periods_df_2 = pd.DataFrame(index=currency_forecast_4_periods_df.index)

# Loop through each currency pair in the forecast DataFrame
for currency_pair in currency_forecast_4_periods_df.index:
    # Extract the base/quote from the first 7 characters
    base_quote = currency_pair[:7]
    
    # Ensure the base/quote exists in merged_dfs
    if base_quote in merged_dfs.columns:
        # Get the last value of the base/quote from merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]
        
        # Get the weighted result for the currency pair from currency_forecast_1_period_df
        weighted_result = currency_forecast_4_periods_df.loc[currency_pair]
        
        # Calculate the updated value
        updated_value = (last_value * weighted_result) + last_value
        
        # Add the updated value to the new DataFrame
        currency_forecast_4_periods_df_2.loc[currency_pair, 'Rates Forecast'] = updated_value
    else:
        print(f"Base/Quote {base_quote} does not exist in merged_dfs. Skipping.")

# Display the final DataFrame
currency_forecast_4_periods_df_2

,Rates Forecast
CHF/CAD 4 Periods,1.645809
EUR/CAD 4 Periods,1.517026
GBP/CAD 4 Periods,1.763847
USD/CAD 4 Periods,1.302149
CAD/CHF 4 Periods,0.610583
EUR/CHF 4 Periods,0.924560
GBP/CHF 4 Periods,1.098396
USD/CHF 4 Periods,0.842747
CAD/EUR 4 Periods,0.663111
CHF/EUR 4 Periods,1.083595


In [45]:
# Add a new column for percentage changes to the existing DataFrame
currency_forecast_4_periods_df_2['Percentage Change'] = None

# Loop through each currency pair
for currency_pair in currency_forecast_4_periods_df_2.index:
    base_quote = currency_pair[:7]  # Extract base/quote (first 7 characters of the index)
    
    if base_quote in merged_dfs.columns:  # Check if the column exists in merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]  # Get the last value from merged_dfs
        rates_forecast = currency_forecast_4_periods_df_2.loc[currency_pair, 'Rates Forecast']  # Get the forecast value
        
        # Calculate the percentage change
        percentage_change = ((rates_forecast - last_value) / last_value) * 100
        
        # Store the percentage change in the existing DataFrame
        currency_forecast_4_periods_df_2.at[currency_pair, 'Percentage Change'] = percentage_change
    else:
        print(f"Column {base_quote} does not exist in merged_dfs. Skipping.")

# Display the updated DataFrame
currency_forecast_4_periods_df_2

,Rates Forecast,Percentage Change
CHF/CAD 4 Periods,1.645809,2.400081
EUR/CAD 4 Periods,1.517026,0.537996
GBP/CAD 4 Periods,1.763847,-2.437915
USD/CAD 4 Periods,1.302149,-3.607367
CAD/CHF 4 Periods,0.610583,-1.865063
EUR/CHF 4 Periods,0.924560,-1.519273
GBP/CHF 4 Periods,1.098396,-2.35314
USD/CHF 4 Periods,0.842747,0.267388
CAD/EUR 4 Periods,0.663111,0.057329
CHF/EUR 4 Periods,1.083595,1.730359


In [46]:
# Right=first df, left=second df
# Outer=ensures that all rows from both DataFrames are included (rows with missing values in one DataFrame will have NaN)
currency_forecast_4_periods_df_2 = pd.merge(
    currency_forecast_4_periods_df_2,
    regression_results_4_periods_df[['R-squared']],  # Select only the 'R-squared' column
    left_index=True,
    right_index=True,
    how='inner'  # Use 'inner' or any other join type you prefer
)

currency_forecast_4_periods_df_2

,Rates Forecast,Percentage Change,R-squared
CHF/CAD 4 Periods,1.645809,2.400081,0.110129
EUR/CAD 4 Periods,1.517026,0.537996,0.091633
GBP/CAD 4 Periods,1.763847,-2.437915,0.067970
USD/CAD 4 Periods,1.302149,-3.607367,0.406512
CAD/CHF 4 Periods,0.610583,-1.865063,0.117479
EUR/CHF 4 Periods,0.924560,-1.519273,0.106381
GBP/CHF 4 Periods,1.098396,-2.35314,0.088170
USD/CHF 4 Periods,0.842747,0.267388,0.064725
CAD/EUR 4 Periods,0.663111,0.057329,0.089247
CHF/EUR 4 Periods,1.083595,1.730359,0.098055


In [47]:
# Update the index to only include the first 7 characters
currency_forecast_4_periods_df_2.index = currency_forecast_4_periods_df_2.index.str[:7]

# Rename the column
currency_forecast_4_periods_df_2.columns = ['Forecast (1Y)', 'Change (1Y)', 'R-squared (1Y)']

# Change numbers format for column 'Change (4M)' to include %
currency_forecast_4_periods_df_2['Change (1Y)'] = currency_forecast_4_periods_df_2['Change (1Y)'].apply(lambda x: f"{x:.2f}%")

# Display the updated DataFrame
currency_forecast_4_periods_df_2

,Forecast (1Y),Change (1Y),R-squared (1Y)
CHF/CAD,1.645809,2.40%,0.110129
EUR/CAD,1.517026,0.54%,0.091633
GBP/CAD,1.763847,-2.44%,0.067970
USD/CAD,1.302149,-3.61%,0.406512
CAD/CHF,0.610583,-1.87%,0.117479
EUR/CHF,0.924560,-1.52%,0.106381
GBP/CHF,1.098396,-2.35%,0.088170
USD/CHF,0.842747,0.27%,0.064725
CAD/EUR,0.663111,0.06%,0.089247
CHF/EUR,1.083595,1.73%,0.098055


### 8 Periods

In [48]:
# Filter columns ending with '8 Period'
dependent_vars_8_periods = [col for col in final_df.columns if col.endswith('8 Periods')]

# Prepare a list to store regression results
regression_results_8_periods = []

# Loop through dependent variables dynamically
for y_col in dependent_vars_8_periods:
    # Extract currency pair identifiers
    base, quote = y_col.split('/')[0], y_col.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    if apr_col in final_df.columns and cpi_col in final_df.columns and gdp_base_col in final_df.columns and gdp_quote_col in final_df.columns:
        # Prepare data for regression
        X = final_df[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]].dropna()
        Y = final_df[y_col].dropna()
        
        # Align indexes
        aligned_data = pd.concat([X, Y], axis=1).dropna()
        X = aligned_data[[apr_col, cpi_col, gdp_base_col, gdp_quote_col]]
        Y = aligned_data[y_col]
        
        # Add constant to X
        X = sm.add_constant(X)
        
        # Run regression
        model = sm.OLS(Y, X).fit()
        intercept = model.params['const']
        apr_beta = model.params[apr_col]
        cpi_beta = model.params[cpi_col]
        gdp_base_beta = model.params[gdp_base_col]
        gdp_quote_beta = model.params[gdp_quote_col]
        r_squared = model.rsquared
        
        # Append the results to the list
        regression_results_8_periods.append({
            'Currency Pair': y_col,
            'Intercept': intercept,
            'APR Beta': apr_beta,
            'CPI Beta': cpi_beta,
            'GDP Base Beta': gdp_base_beta,
            'GDP Quote Beta': gdp_quote_beta,
            'R-squared': r_squared
        })

# Convert the list of results to a DataFrame
regression_results_8_periods_df = pd.DataFrame(regression_results_8_periods)

# Set the 'Currency Pair' column as the index
regression_results_8_periods_df.set_index('Currency Pair', inplace=True)

# Display the DataFrame
regression_results_8_periods_df

,Intercept,APR Beta,CPI Beta,GDP Base Beta,GDP Quote Beta,R-squared
Currency Pair,,,,,,
CHF/CAD 8 Periods,0.105024,1.294702,-0.241047,-2.034671,0.107927,0.241105
EUR/CAD 8 Periods,0.018890,1.241220,-2.172401,-0.037258,-0.689201,0.154024
GBP/CAD 8 Periods,0.014779,-2.293062,-3.269873,1.341310,-2.411843,0.234660
USD/CAD 8 Periods,0.013101,-2.747859,-6.217074,3.570699,-3.073449,0.426214
CAD/CHF 8 Periods,-0.088714,1.224673,-0.394362,-0.154639,1.957369,0.237785
EUR/CHF 8 Periods,-0.041078,0.418211,-1.875749,-0.202251,1.237260,0.207564
GBP/CHF 8 Periods,-0.106632,1.608595,-0.838955,0.028186,1.651044,0.251176
USD/CHF 8 Periods,-0.056325,1.567895,-0.400888,0.704320,-0.325537,0.126100
CAD/EUR 8 Periods,-0.010446,0.884227,-2.303371,0.921384,-0.086190,0.150676


In [49]:
# Create a DataFrame to store the weighted results
currency_forecast_8_periods_df = pd.DataFrame(index=final_df.index)

# Loop through the regression results
for currency_pair, params in regression_results_8_periods_df.iterrows():
    # Extract the base and quote currencies
    base, quote = currency_pair.split('/')[0], currency_pair.split('/')[1].split()[0]
    
    # Find the matching explanatory variable columns
    apr_col = f"{base} APR-{quote} APR"
    cpi_col = f"{base} CPI-{quote} CPI"
    gdp_base_col = f"{base} GDP"
    gdp_quote_col = f"{quote} GDP"
    
    # Check if all necessary columns exist in final_df
    if all(col in final_df.columns for col in [apr_col, cpi_col, gdp_base_col, gdp_quote_col]):
        # Get the most recent data for the explanatory variables
        most_recent_data = final_df.iloc[-1]
        
        # Calculate the weighted result for this currency pair
        weighted_result = (
            params['Intercept'] * 1 +  # Intercept term
            params['APR Beta'] * most_recent_data[apr_col] +
            params['CPI Beta'] * most_recent_data[cpi_col] +
            params['GDP Base Beta'] * most_recent_data[gdp_base_col] +
            params['GDP Quote Beta'] * most_recent_data[gdp_quote_col]
        )
        
        # Store the weighted result in the DataFrame
        currency_forecast_8_periods_df.loc[most_recent_data.name, currency_pair] = weighted_result
    else:
        print(f"Missing one or more columns for {currency_pair}. Skipping.")

currency_forecast_8_periods_df = currency_forecast_8_periods_df.iloc[-1]
currency_forecast_8_periods_df

CHF/CAD 8 Periods    0.023805
EUR/CAD 8 Periods   -0.000498
GBP/CAD 8 Periods   -0.047913
USD/CAD 8 Periods   -0.001040
CAD/CHF 8 Periods   -0.013243
EUR/CHF 8 Periods   -0.024825
GBP/CHF 8 Periods   -0.021802
USD/CHF 8 Periods    0.020344
CAD/EUR 8 Periods    0.008165
CHF/EUR 8 Periods    0.030685
GBP/EUR 8 Periods   -0.021874
USD/EUR 8 Periods    0.029448
CAD/GBP 8 Periods    0.061141
CHF/GBP 8 Periods    0.026922
EUR/GBP 8 Periods    0.026132
USD/GBP 8 Periods    0.021269
CAD/USD 8 Periods    0.008832
CHF/USD 8 Periods   -0.022444
EUR/USD 8 Periods   -0.023847
GBP/USD 8 Periods   -0.014495
Name: 2024-09, dtype: float64

In [50]:
# Create a new DataFrame for the updated values
currency_forecast_8_periods_df_2 = pd.DataFrame(index=currency_forecast_8_periods_df.index)

# Loop through each currency pair in the forecast DataFrame
for currency_pair in currency_forecast_8_periods_df.index:
    # Extract the base/quote from the first 7 characters
    base_quote = currency_pair[:7]
    
    # Ensure the base/quote exists in merged_dfs
    if base_quote in merged_dfs.columns:
        # Get the last value of the base/quote from merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]
        
        # Get the weighted result for the currency pair from currency_forecast_1_period_df
        weighted_result = currency_forecast_8_periods_df.loc[currency_pair]
        
        # Calculate the updated value
        updated_value = (last_value * weighted_result) + last_value
        
        # Add the updated value to the new DataFrame
        currency_forecast_8_periods_df_2.loc[currency_pair, 'Rates Forecast'] = updated_value
    else:
        print(f"Base/Quote {base_quote} does not exist in merged_dfs. Skipping.")

# Display the final DataFrame
currency_forecast_8_periods_df_2

,Rates Forecast
CHF/CAD 8 Periods,1.645494
EUR/CAD 8 Periods,1.508157
GBP/CAD 8 Periods,1.721300
USD/CAD 8 Periods,1.349475
CAD/CHF 8 Periods,0.613947
EUR/CHF 8 Periods,0.915517
GBP/CHF 8 Periods,1.100342
USD/CHF 8 Periods,0.857599
CAD/EUR 8 Periods,0.668142
CHF/EUR 8 Periods,1.097848


In [51]:
# Add a new column for percentage changes to the existing DataFrame
currency_forecast_8_periods_df_2['Percentage Change'] = None

# Loop through each currency pair
for currency_pair in currency_forecast_8_periods_df_2.index:
    base_quote = currency_pair[:7]  # Extract base/quote (first 7 characters of the index)
    
    if base_quote in merged_dfs.columns:  # Check if the column exists in merged_dfs
        last_value = merged_dfs[base_quote].iloc[-1]  # Get the last value from merged_dfs
        rates_forecast = currency_forecast_8_periods_df_2.loc[currency_pair, 'Rates Forecast']  # Get the forecast value
        
        # Calculate the percentage change
        percentage_change = ((rates_forecast - last_value) / last_value) * 100
        
        # Store the percentage change in the existing DataFrame
        currency_forecast_8_periods_df_2.at[currency_pair, 'Percentage Change'] = percentage_change
    else:
        print(f"Column {base_quote} does not exist in merged_dfs. Skipping.")

# Display the updated DataFrame
currency_forecast_8_periods_df_2

,Rates Forecast,Percentage Change
CHF/CAD 8 Periods,1.645494,2.380469
EUR/CAD 8 Periods,1.508157,-0.049757
GBP/CAD 8 Periods,1.721300,-4.791283
USD/CAD 8 Periods,1.349475,-0.104028
CAD/CHF 8 Periods,0.613947,-1.324311
EUR/CHF 8 Periods,0.915517,-2.482511
GBP/CHF 8 Periods,1.100342,-2.180194
USD/CHF 8 Periods,0.857599,2.034376
CAD/EUR 8 Periods,0.668142,0.816527
CHF/EUR 8 Periods,1.097848,3.068469


In [52]:
# Right=first df, left=second df
# Outer=ensures that all rows from both DataFrames are included (rows with missing values in one DataFrame will have NaN)
currency_forecast_8_periods_df_2 = pd.merge(
    currency_forecast_8_periods_df_2,
    regression_results_8_periods_df[['R-squared']],  # Select only the 'R-squared' column
    left_index=True,
    right_index=True,
    how='inner'  # Use 'inner' or any other join type you prefer
)

currency_forecast_8_periods_df_2

,Rates Forecast,Percentage Change,R-squared
CHF/CAD 8 Periods,1.645494,2.380469,0.241105
EUR/CAD 8 Periods,1.508157,-0.049757,0.154024
GBP/CAD 8 Periods,1.721300,-4.791283,0.234660
USD/CAD 8 Periods,1.349475,-0.104028,0.426214
CAD/CHF 8 Periods,0.613947,-1.324311,0.237785
EUR/CHF 8 Periods,0.915517,-2.482511,0.207564
GBP/CHF 8 Periods,1.100342,-2.180194,0.251176
USD/CHF 8 Periods,0.857599,2.034376,0.126100
CAD/EUR 8 Periods,0.668142,0.816527,0.150676
CHF/EUR 8 Periods,1.097848,3.068469,0.193367


In [53]:
# Update the index to only include the first 7 characters
currency_forecast_8_periods_df_2.index = currency_forecast_8_periods_df_2.index.str[:7]

# Rename the column
currency_forecast_8_periods_df_2.columns = ['Forecast (2Y)', 'Change (2Y)', 'R-squared (2Y)']

# Change numbers format for column 'Change (4M)' to include %
currency_forecast_8_periods_df_2['Change (2Y)'] = currency_forecast_8_periods_df_2['Change (2Y)'].apply(lambda x: f"{x:.2f}%")

# Display the updated DataFrame
currency_forecast_8_periods_df_2

,Forecast (2Y),Change (2Y),R-squared (2Y)
CHF/CAD,1.645494,2.38%,0.241105
EUR/CAD,1.508157,-0.05%,0.154024
GBP/CAD,1.721300,-4.79%,0.234660
USD/CAD,1.349475,-0.10%,0.426214
CAD/CHF,0.613947,-1.32%,0.237785
EUR/CHF,0.915517,-2.48%,0.207564
GBP/CHF,1.100342,-2.18%,0.251176
USD/CHF,0.857599,2.03%,0.126100
CAD/EUR,0.668142,0.82%,0.150676
CHF/EUR,1.097848,3.07%,0.193367


### Merge Currency Forecast dfs

In [54]:
# Right=first df, left=second df
# Outer=ensures that all rows from both DataFrames are included (rows with missing values in one DataFrame will have NaN)
final_currency_forecast_df = pd.merge(currency_forecast_1_period_df_2, currency_forecast_4_periods_df_2, left_index=True, right_index=True, how='inner')
final_currency_forecast_df = pd.merge(final_currency_forecast_df, currency_forecast_8_periods_df_2, left_index=True, right_index=True, how='inner')

# Get the last date from the DataFrame index
last_date = final_df.index[-1]

# Print a guiding message
print(f"The forecasts are based on data up to {last_date}. The 3-month, 1-year, and 2-year forecasts are calculated starting from this date.\n")

final_currency_forecast_df

The forecasts are based on data up to 2024-09. The 3-month, 1-year, and 2-year forecasts are calculated starting from this date.



,Forecast (3M),Change (3M),R-squared (3M),Forecast (1Y),Change (1Y),R-squared (1Y),Forecast (2Y),Change (2Y),R-squared (2Y)
CHF/CAD,1.642919,2.22%,0.038472,1.645809,2.40%,0.110129,1.645494,2.38%,0.241105
EUR/CAD,1.512275,0.22%,0.035006,1.517026,0.54%,0.091633,1.508157,-0.05%,0.154024
GBP/CAD,1.794538,-0.74%,0.060921,1.763847,-2.44%,0.067970,1.721300,-4.79%,0.234660
USD/CAD,1.325191,-1.90%,0.115359,1.302149,-3.61%,0.406512,1.349475,-0.10%,0.426214
CAD/CHF,0.610029,-1.95%,0.038017,0.610583,-1.87%,0.117479,0.613947,-1.32%,0.237785
EUR/CHF,0.931756,-0.75%,0.034699,0.924560,-1.52%,0.106381,0.915517,-2.48%,0.207564
GBP/CHF,1.117841,-0.62%,0.021572,1.098396,-2.35%,0.088170,1.100342,-2.18%,0.251176
USD/CHF,0.832782,-0.92%,0.030273,0.842747,0.27%,0.064725,0.857599,2.03%,0.126100
CAD/EUR,0.662577,-0.02%,0.034729,0.663111,0.06%,0.089247,0.668142,0.82%,0.150676
CHF/EUR,1.074929,0.92%,0.030401,1.083595,1.73%,0.098055,1.097848,3.07%,0.193367


In [55]:
# Sort in alphabetical order
final_currency_forecast_df = final_currency_forecast_df.sort_index()

# Print a guiding message
print(f"The forecasts are based on data up to {last_date}. The 3-month, 1-year, and 2-year forecasts are calculated starting from this date.\n")

final_currency_forecast_df

The forecasts are based on data up to 2024-09. The 3-month, 1-year, and 2-year forecasts are calculated starting from this date.



,Forecast (3M),Change (3M),R-squared (3M),Forecast (1Y),Change (1Y),R-squared (1Y),Forecast (2Y),Change (2Y),R-squared (2Y)
CAD/CHF,0.610029,-1.95%,0.038017,0.610583,-1.87%,0.117479,0.613947,-1.32%,0.237785
CAD/EUR,0.662577,-0.02%,0.034729,0.663111,0.06%,0.089247,0.668142,0.82%,0.150676
CAD/GBP,0.558317,0.94%,0.061648,0.570834,3.20%,0.067247,0.586939,6.11%,0.217856
CAD/USD,0.754660,1.95%,0.111834,0.769129,3.90%,0.376321,0.746796,0.88%,0.373903
CHF/CAD,1.642919,2.22%,0.038472,1.645809,2.40%,0.110129,1.645494,2.38%,0.241105
CHF/EUR,1.074929,0.92%,0.030401,1.083595,1.73%,0.098055,1.097848,3.07%,0.193367
CHF/GBP,0.896554,0.85%,0.013402,0.914463,2.86%,0.083538,0.912928,2.69%,0.247726
CHF/USD,1.201671,1.00%,0.031300,1.187388,-0.20%,0.079417,1.163065,-2.24%,0.142684
EUR/CAD,1.512275,0.22%,0.035006,1.517026,0.54%,0.091633,1.508157,-0.05%,0.154024
EUR/CHF,0.931756,-0.75%,0.034699,0.924560,-1.52%,0.106381,0.915517,-2.48%,0.207564
